In [22]:
import os
import glob
import numpy as np


import keras
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv2DTranspose, Activation, concatenate
from keras.optimizers import Adam
from keras import losses
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K

import cv2
import tensorflow as tf
from scipy.misc import toimage
from PIL import Image

In [23]:

def getImageArr( path , width , height , imgNorm="sub_mean" , odering='channels_last' ):

    try:
        img = Image.open(path)
        
        if imgNorm == "sub_and_divide":
            img = np.float32(cv2.resize(img, ( width , height ))) / 127.5 - 1
        elif imgNorm == "sub_mean":
            img = cv2.resize(img, ( width , height ))
            img = img.astype(np.float32)
            img[:,:,0] -= 103.939
            img[:,:,1] -= 116.779
            img[:,:,2] -= 123.68
        elif imgNorm == "divide":
            img = cv2.resize(img, ( width , height ))
            img = img.astype(np.float32)
            img = img/255.0

        if odering == 'channels_first':
            img = np.rollaxis(img, 2, 0)
        return img
    except Exception, e:
        print path , e
        img = np.zeros((  height , width  , 3 ))
        if odering == 'channels_first':
            img = np.rollaxis(img, 2, 0)
        return img

def getSegmentationArr( path , nClasses ,  width , height  ):

    seg_labels = np.zeros((  height , width  , nClasses ))
    try:
        img = Image.open(path)
        img = cv2.resize(img, ( width , height ))
        img = img[:, : , 0]

        for c in range(nClasses):
            seg_labels[: , : , c ] = (img == c ).astype(int)

    except Exception, e:
        print e

    seg_labels = np.reshape(seg_labels, ( width*height , nClasses ))
    return seg_labels

def imageSegmentationGenerator( images_path , segs_path ,  batch_size,  n_classes , input_height , input_width , output_height , output_width   ):
    
    assert images_path[-1] == '/'
    assert segs_path[-1] == '/'

    images = glob.glob( images_path + "*.jpg"  ) + glob.glob( images_path + "*.png"  ) +  glob.glob( images_path + "*.jpeg"  )
    images.sort()
    segmentations  = glob.glob( segs_path + "*.jpg"  ) + glob.glob( segs_path + "*.png"  ) +  glob.glob( segs_path + "*.jpeg"  )
    segmentations.sort()

    assert len( images ) == len(segmentations)
    for im , seg in zip(images,segmentations):
        assert(  im.split('/')[-1].split(".")[0] ==  seg.split('/')[-1].split(".")[0] )

    zipped = itertools.cycle( zip(images,segmentations) )

    while True:
        X = []
        Y = []
        for _ in range( batch_size) :
            im , seg = zipped.next()
            X.append( getImageArr(im , input_width , input_height )  )
            Y.append( getSegmentationArr( seg , n_classes , output_width , output_height )  )

        yield np.array(X) , np.array(Y)

In [24]:
def new_cross_entropy(y_, y_conv):
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
    return cross_entropy
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def dice(y_true, y_pred):
    y_true_f = K.clip(K.batch_flatten(y_true), 0., 1.)
    y_pred_f = K.clip(K.batch_flatten(y_pred), 0., 1.)

    intersection = 2 * K.sum(y_true_f * y_pred_f, axis=1)
    union = K.sum(y_true_f * y_true_f, axis=1) + K.sum(y_pred_f * y_pred_f, axis=1)
    return K.mean(intersection / union)


def dice_loss(y_true, y_pred):
    return -dice(y_true, y_pred)


def log_dice_loss(y_true, y_pred):
    return -K.log(dice(y_true, y_pred))


def dice_metric(y_true, y_pred):
    """An exact Dice score for binary tensors."""
    y_true_f = K.greater(y_true, 0.5)
    y_pred_f = K.greater(y_pred, 0.5)
    return dice(y_true_f, y_pred_f)


In [25]:
K.set_image_dim_ordering('tf')

In [26]:
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols, 3), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        arr = imgs[i]
        arr = arr.astype('float32')
        arr /= 255.
        imgs_p[i] = resize(arr, (img_cols, img_rows), preserve_range=True)
    return imgs_p

In [27]:
#imgs_cup_train = np.load('gt_cup_train_new.npy')
#imgs_orig_train = np.load('orig_cup_train_new.npy')

In [28]:
def get_model_3():
    model_vgg19_conv = VGG19(weights='imagenet', include_top=False, input_shape=(256,256,3))

    conv2_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv2_3_16", data_format="channels_last")(model_vgg19_conv.layers[-17].output)

    conv3_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv3_3_16", data_format="channels_last")(model_vgg19_conv.layers[-12].output)

    conv4_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv4_3_16", data_format="channels_last")(model_vgg19_conv.layers[-7].output)

    conv5_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv5_3_16", data_format="channels_last")(model_vgg19_conv.layers[-2].output)

    upsample2_ = Conv2DTranspose(filters=16, kernel_size=(4, 4), strides=(2, 2), padding = "same", name= "upsample2_", data_format="channels_last")(conv2_3_16)
    
    upsample4_ = Conv2DTranspose(filters=16, kernel_size=(8, 8), strides=(4, 4), padding = "same", name= "upsample4_", data_format="channels_last")(conv3_3_16)

    upsample8_ = Conv2DTranspose(filters=16, kernel_size=(16, 16), strides=(8, 8), padding = "same",  name= "upsample8_", data_format="channels_last")(conv4_3_16)

    upsample16_ = Conv2DTranspose(filters=16, kernel_size=(32, 32), strides=(16, 16), padding = "same", name= "upsample16_", data_format="channels_last")(conv5_3_16)

    concat_upscore = concatenate([upsample2_, upsample4_, upsample8_, upsample16_], axis=-1)
    
    """
    new_score_weighting = Conv2D(filters=1, kernel_size=(1,1), activation='sigmoid', kernel_initializer=keras.initializers.Constant(value=0.01), name= "new_score_weighting", data_format="channels_last")(concat_upscore)
    
    my_model = Model(input=model_vgg19_conv.input , output=new_score_weighting)
    
    """

    conv6 = Conv2D(filters=1, kernel_size=(1,1), activation='relu', kernel_initializer=keras.initializers.Constant(value=0.01), name= "conv6", data_format="channels_last", border_mode='same')(concat_upscore)
    conv6 = Reshape((1,256*256))(conv6)
    conv6 = Permute((2,1))(conv6)

    conv7 = Activation('softmax')(conv6)

    my_model = Model(input=model_vgg19_conv.input, output=conv7)

    return my_model

In [29]:
from keras.models import Sequential
from keras.layers import Reshape
from keras.models import Model
from keras.layers import Layer, Dense, Dropout, Activation, Flatten, Reshape, concatenate, Permute
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D , ZeroPadding3D , UpSampling3D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.layers.recurrent import LSTM
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam , SGD
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from keras import backend as K





def get_model_2(nClasses , optimizer=None , input_width=256 , input_height=256 , nChannels=3): 
    
    inputs = Input((input_height, input_width, nChannels))
    conv1 = Convolution2D(32, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Convolution2D(32, kernel_size=(3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Convolution2D(64, kernel_size=(3, 3), activation='relu', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Convolution2D(64, kernel_size=(3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Convolution2D(128, kernel_size=(3, 3), activation='relu', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Convolution2D(128, kernel_size=(3, 3), activation='relu', padding='same')(conv3)

    up1 = concatenate([UpSampling2D(size=(2, 2))(conv3), conv2], axis=-1)
    conv4 = Convolution2D(64, kernel_size=(3, 3), activation='relu', padding='same')(up1)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Convolution2D(64, kernel_size=(3, 3), activation='relu', padding='same')(conv4)
    
    up2 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv1], axis=-1)
    conv5 = Convolution2D(32, kernel_size=(3, 3), activation='relu', padding='same')(up2)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Convolution2D(32, kernel_size=(3, 3), activation='relu', padding='same')(conv5)
    
    conv6 = Convolution2D(nClasses, kernel_size=(1, 1), activation='relu',padding='same')(conv5)
    conv6 = core.Reshape((nClasses,input_height*input_width))(conv6)
    conv6 = core.Permute((2,1))(conv6)


    conv7 = core.Activation('softmax')(conv6)

    model = Model(input=inputs, output=conv7)

    return model
    
    

In [30]:
def get_model():
    model_vgg19_conv = VGG19(weights='imagenet', include_top=False, input_shape=(256,256,3))

    conv2_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv2_3_16", data_format="channels_last")(model_vgg19_conv.layers[-17].output)

    conv3_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv3_3_16", data_format="channels_last")(model_vgg19_conv.layers[-12].output)

    conv4_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv4_3_16", data_format="channels_last")(model_vgg19_conv.layers[-7].output)

    conv5_3_16 = Conv2D(filters=16, kernel_size=(3, 3), padding = "same", kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.001), name= "conv5_3_16", data_format="channels_last")(model_vgg19_conv.layers[-2].output)

    upsample2_ = Conv2DTranspose(filters=16, kernel_size=(4, 4), strides=(2, 2), padding = "same", name= "upsample2_", data_format="channels_last")(conv2_3_16)
    
    upsample4_ = Conv2DTranspose(filters=16, kernel_size=(8, 8), strides=(4, 4), padding = "same", name= "upsample4_", data_format="channels_last")(conv3_3_16)

    upsample8_ = Conv2DTranspose(filters=16, kernel_size=(16, 16), strides=(8, 8), padding = "same",  name= "upsample8_", data_format="channels_last")(conv4_3_16)

    upsample16_ = Conv2DTranspose(filters=16, kernel_size=(32, 32), strides=(16, 16), padding = "same", name= "upsample16_", data_format="channels_last")(conv5_3_16)

    concat_upscore = concatenate([upsample2_, upsample4_, upsample8_, upsample16_], axis=-1)
    
    new_score_weighting = Conv2D(filters=1, kernel_size=(1,1), activation='sigmoid', kernel_initializer=keras.initializers.Constant(value=0.01), name= "new_score_weighting", data_format="channels_last")(concat_upscore)
    
    my_model = Model(input=model_vgg19_conv.input , output=new_score_weighting)

    return my_model

In [31]:
model = get_model()

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ne..., inputs=Tensor("in...)`


In [14]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 128, 128, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [32]:
model.load_weights("log_dice_weights_200_iter.h5")
adam = Adam(lr=.000001)

In [70]:
model.compile(optimizer=adam, loss=tf.losses.sigmoid_cross_entropy)
model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

In [20]:
model.fit(x=imgs_orig_train, y=imgs_cup_train, batch_size=16, epochs=40, shuffle=True, validation_split=0.2, callbacks=[model_checkpoint])           

NameError: name 'imgs_orig_train' is not defined

In [33]:
import random

In [36]:
colors = [  ( random.randint(0,255),random.randint(0,255),random.randint(0,255)   ) for _ in range(1)  ]
X = getImageArr("31.jpg" , 256  , 256, "divide"  )
X = np.expand_dims(X, 0)
pr = model.predict( np.array(X) )
pr = pr.reshape(( 256 ,  256 , 1 ) ).argmax( axis=2 )
seg_img = np.zeros( ( 256 , 256 , 3  ) )
for c in range(1):
    seg_img[:,:,0] += ( (pr[:,: ] == c )*( colors[c][0] )).astype('uint8')
    seg_img[:,:,1] += ((pr[:,: ] == c )*( colors[c][1] )).astype('uint8')
    seg_img[:,:,2] += ((pr[:,: ] == c )*( colors[c][2] )).astype('uint8')
seg_img = cv2.resize(seg_img  , (256 , 256 ))
cv2.imwrite(  "final.jpg" , seg_img )

31.jpg src is not a numpy array, neither a scalar


True

In [37]:
cv2.imwrite(  "final2.jpg" , pr.astype('uint8'))

True

In [11]:
new_img = Image.open("31.jpg")
if new_img.mode != 'RGB':
    new_img = new_img.convert('RGB')
# new_img.show()
new_arr = np.array(new_img)
new_arr = new_arr/255.
new_arr = np.expand_dims(new_arr, axis = 0)
print(new_arr.shape)

(1, 256, 256, 3)


In [12]:
prediction = model.predict(new_arr)
prediction = prediction*255.

In [28]:
prediction = pr

In [29]:
prediction = np.squeeze(prediction)
if prediction.shape == (65536,):
    prediction = np.reshape(prediction, (256,256))
print(prediction.shape)

(256, 256)


In [30]:
pred_img = Image.fromarray(prediction)
if pred_img.mode != 'RGB':
    pred_img = pred_img.convert('RGB')
pred_img.show()

In [8]:

import numpy as np
import cv2
import glob


def getImageArr( path , width , height , imgNorm="sub_mean" , odering='channels_first' ):

    try:
        img = cv2.imread(path, 1)

        if imgNorm == "sub_and_divide":
            img = np.float32(cv2.resize(img, ( width , height ))) / 127.5 - 1
        elif imgNorm == "sub_mean":
            img = cv2.resize(img, ( width , height ))
            img = img.astype(np.float32)
            img[:,:,0] -= 103.939
            img[:,:,1] -= 116.779
            img[:,:,2] -= 123.68
        elif imgNorm == "divide":
            img = cv2.resize(img, ( width , height ))
            img = img.astype(np.float32)
            img = img/255.0

        if odering == 'channels_first':
            img = np.rollaxis(img, 2, 0)
        return img
    except Exception, e:
        print path , e
        img = np.zeros((  height , width  , 3 ))
        if odering == 'channels_first':
            img = np.rollaxis(img, 2, 0)
        return img

def getSegmentationArr( path , nClasses ,  width , height  ):

    seg_labels = np.zeros((  height , width  , nClasses ))
    try:
        img = cv2.imread(path, 1)
        img = cv2.resize(img, ( width , height ))
        img = img[:, : , 0]

        for c in range(nClasses):
            seg_labels[: , : , c ] = (img == c ).astype(int)

    except Exception, e:
        print e

    seg_labels = np.reshape(seg_labels, ( width*height , nClasses ))
    return seg_labels

def imageSegmentationGenerator( images_path , segs_path ,  batch_size,  n_classes , input_height , input_width , output_height , output_width   ):
    
    assert images_path[-1] == '/'
    assert segs_path[-1] == '/'

    images = glob.glob( images_path + "*.jpg"  ) + glob.glob( images_path + "*.png"  ) +  glob.glob( images_path + "*.jpeg"  )
    images.sort()
    segmentations  = glob.glob( segs_path + "*.jpg"  ) + glob.glob( segs_path + "*.png"  ) +  glob.glob( segs_path + "*.jpeg"  )
    segmentations.sort()

    assert len( images ) == len(segmentations)
    for im , seg in zip(images,segmentations):
        assert(  im.split('/')[-1].split(".")[0] ==  seg.split('/')[-1].split(".")[0] )

    zipped = itertools.cycle( zip(images,segmentations) )

    while True:
        X = []
        Y = []
        for _ in range( batch_size) :
            im , seg = zipped.next()
            X.append( getImageArr(im , input_width , input_height )  )
            Y.append( getSegmentationArr( seg , n_classes , output_width , output_height )  )

        yield np.array(X) , np.array(Y)


# import Models , LoadBatches
# G  = LoadBatches.imageSegmentationGenerator( "data/clothes_seg/prepped/images_prepped_train/" ,  "data/clothes_seg/prepped/annotations_prepped_train/" ,  1,  10 , 800 , 550 , 400 , 272   ) 
# G2  = LoadBatches.imageSegmentationGenerator( "data/clothes_seg/prepped/images_prepped_test/" ,  "data/clothes_seg/prepped/annotations_prepped_test/" ,  1,  10 , 800 , 550 , 400 , 272   ) 

# m = Models.VGGSegnet.VGGSegnet( 10  , use_vgg_weights=True ,  optimizer='adadelta' , input_image_size=( 800 , 550 )  )
# m.fit_generator( G , 512  , nb_epoch=10 )

In [9]:
training_gt_path = "/Users/karimabedrabbo/Desktop/Version 18 - Training Cup Segementation CNN/cropped_augmentations_unblurred_resized/train/cup"
training_orig_path = "/Users/karimabedrabbo/Desktop/Version 18 - Training Cup Segementation CNN/cropped_augmentations_unblurred_resized/train/norm"
cup_train_data_gen = imageSegmentationGenerator(training_orig_path, training_gt_path, 16, 2, 256, 256, 256, 256)

In [65]:
unet_masks = np.load("imgs_mask_test.npy")

In [66]:
unet_masks = unet_masks * 50.

In [67]:
img = Image.fromarray(np.squeeze(unet_masks[5]))

In [68]:
img.show()